In [1]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import as_strided
from NilsPodLib import session as sensor
from datetime import datetime, timedelta
import scipy.signal as ss
from scipy.fftpack import fft
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.formatters import DatetimeTickFormatter

output_notebook()

Loading BokehJS ...

In [2]:
def get_windows(arr, window, overlap):
    window_step = window - overlap
    new_shape = arr.shape[:-1] + ((arr.shape[-1] - overlap) // window_step,
                                  window)
    new_strides = (arr.strides[:-1] + (window_step * arr.strides[-1],) +
                   arr.strides[-1:])
    arr_new = as_strided(arr, shape=new_shape, strides=new_strides)
    return arr_new

In [3]:
def markArea(ax, color , values, idx):
    ax.axvspan(values[idx[0]],values[idx[1]], facecolor = color, alpha = 0.1)
    for i in range(1,len(idx)-2):
        ax.axvspan(values[idx[i]+1],values[idx[i+1]], facecolor = color, alpha = 0.1)

In [4]:
sensor_id = "3388"
date = "20190406_0853"
folder_path = "./Data/Team_ArneRobert/"
file_name = "NilsPodX-%s_%s.csv" % (sensor_id, date)

file_path = folder_path + file_name
file_path

'./Data/Team_ArneRobert/NilsPodX-3388_20190406_0853.csv'

In [5]:
nilspod_data = pd.read_csv(file_path)

In [6]:
baro_filtered = ss.savgol_filter(nilspod_data['baro'], window_length=1201, polyorder=5)

/Users/Richer/anaconda/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [7]:
baro_window = get_windows(baro_filtered, 200, 0).mean(axis=1)

baro_grad = np.gradient(baro_window)
baro_grad_threshold = np.std(baro_grad)/10

ride = np.where(baro_grad > baro_grad_threshold)[0]
no_ride = np.where(baro_grad < -baro_grad_threshold)[0]

In [12]:
#altitude = 44330 * (1.0 - (baro_window / 1013.0) ** 0.1903)
altitude = 44330 * (1.0 - (baro_window / 1013.0) ** 0.1903)

ref_altitudes = {
    'alp_dado_1': (1948, 300),
    'crest_la_siala_1': (2317, 720),
    'alp_dado_2': (1948, 1500),
    'crest_la_siala_2': (2317, 2080),
    'fuorcla': (2118, 4300),
    'vorab_1': (2570, 5300),
    'vorab_2': (2570, 7100)
}

alt_diffs = list()

for lift in ref_altitudes:
    alt_diffs.append(ref_altitudes[lift][0] - altitude[int(ref_altitudes[lift][1])])

altitude = altitude + np.mean(alt_diffs)

In [13]:
fig_grad = figure(title=date, x_axis_label="Time", y_axis_label="Baro [m]", height=200, sizing_mode='scale_width')
fig_grad.line(range(0,len(baro_window)), -(baro_window - 800)*0.1, legend="Baro", line_color='blue')
fig_grad.line(range(0,len(baro_grad)), baro_grad, legend="Baro Grad", line_color='green')

show(fig_grad)

In [14]:
fig_altitude = figure(title=date, x_axis_label="Time", y_axis_label="Altitude [m]", height=200, sizing_mode='scale_width')
fig_altitude.line(range(0,len(altitude)), altitude, legend="Altitude", line_color='green')
fig_altitude.line(range(0,len(altitude)), 2252, line_color="grey", line_width=2, legend="LAAX Bridge")
fig_altitude.x(ride, 2300, line_color='blue')
fig_altitude.x(no_ride, 2300, line_color='red')
for lift in ref_altitudes:
    fig_altitude.asterisk(ref_altitudes[lift][1], ref_altitudes[lift][0], size=10)
    alt_diffs.append(ref_altitudes[lift][0] - altitude[int(ref_altitudes[lift][1])])
    
show(fig_altitude)